<a href="https://colab.research.google.com/github/KshitijBhat/ML_Enthusiasts_Ishana/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Requires GPU (CUDA). Please connect to a GPU runtime.

Be sure to mount/upload the drive folder `ML_enthsiasts_Ishana`

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import warnings
warnings.filterwarnings("ignore")

import json

!git clone https://github.com/zlckanata/DeepGlobe-Road-Extraction-Challenge.git
!mv DeepGlobe-Road-Extraction-Challenge DeepGlobe
!git clone https://github.com/KshitijBhat/ML_Enthusiasts_Ishana.git
from ML_Enthusiasts_Ishana.Helpers import *

def get_path(image,t, image_path):
    
    kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
    img_sharp = cv2.filter2D(image, ddepth=-1, kernel=kernel)
    out = cv2.addWeighted( image, 1.1, img_sharp, 0.1, -5)
    squares = split_stitch(out)
    
    masks = []
    for cut_img in squares:
        
        mask = solver.test_one_img_from_path(cut_img)
        mask[mask>0.7] = 255
        mask[mask<=0.7] = 0
        mask = np.concatenate([mask[:,:,None],mask[:,:,None],mask[:,:,None]],axis=2)
        masks.append(mask.astype(np.uint8))
    prediction = split_stitch(image,masks=masks)
    points_path = image_path[0:len(image_path)-4] + ".json"

    points = open(points_path) 
    coords = json.load(points)
    start = np.array(coords["Start"])
    end = np.array(coords["End"])

    out = prediction
    out = cv2.cvtColor(out,cv2.COLOR_BGR2GRAY)
    grid = np.ceil((255-out)/255).astype('int')
    
    edges = (cv2.Canny(out, threshold1=100, threshold2=200))/255
    ngrid = narrowize(grid, edges,t)
    G = initgraph(ngrid)
    start = tuple(start[::-1])
    end = tuple(end[::-1])

    astar_path = generate_path(G,start,end,euclidean)

    return astar_path
    
def submit(): 
    ans = {}
    for id in image_ids:
        image = cv2.imread(source+id)
        paths = get_path(image,4,source+id)
        if paths == None:
            paths = get_path(image,0,source+id)
        path = []
        for point in paths:
            (x,y) = point
            path.append([int(y),int(x)])
        ans[id] = path
        print(id,': Done')
    json_object = json.dumps(ans, indent = 4)
    with open("Submission.json", "w") as outfile:
        outfile.write(json_object) 


Upload the 60% images data and JSON files to a folder and fill in the path in the following cell . If it is in a github repo, clone it here and fill in the corresponding path. Example: `source = 'drive/MyDrive/Ishana/Data/'`

JSON file name should be same as of the image.

60% data Folder structure: 
```
Data
  |_____Image1.json
  |_____Image1.png
  |_____Image2.json
  |_____Image2.png
  : ...
  : ...

```




In [ ]:
source =  '<insert path here>' #Like 'drive/MyDrive/Ishana/Data/'
val = os.listdir(source)
image_ids = list()
for filename in val:
  if filename.endswith('.png'):
    image_ids.append(filename)
print(image_ids)    

Unzip the `weights.zip` attached in the folder named `ML_enthusiasts_Ishana` by running the following cell.

Insert the path of the `weights.zip` file.

Example: `model=drive/Shareddrives/ML_enthusiasts_Ishana/weights.zip`

In [ ]:
!model=<insert path here> ; unzip $model
solver = TTAFrame(DinkNet34)
solver.load('log01_dink34.th')

To generate the JSON file for the final submission, run the following cell. The JSON file will be generated as `Submission.json` in the Files section. Download it for evaluation

In [ ]:
submit()